# Programmation orientée objet

Comment créer se propres objets en Python. 

## Vocabulaire

En python tout ce que l'on manipule est un objet : 

* Nombre, liste, dictionnaire, ...
* Tableau numpy
* Fonction, module, ....

Un objet possède des **attributs**. Exemple : 

    z = 1.1J
    z.real
    a = np.array([1, 2, 4])
    a.shape
    
Un objet possède des **méthodes**. Exemple :

    a.mean()
    a.conjugate()
    z.conjugate()
    
Les méthodes sont des fonctions attachées à l'objet. La méthode conjugate d'un tableau n'est pas la même que la méthode conjugate d'un complexe

## Comment créer un objet

### Créer un nombre complexe


In [2]:
z = 1.1J
(1 + 3*z).real

1.0

In [144]:
class Complex:
    def __init__(self, partie_reelle, partie_imaginaire):
        self.re = partie_reelle
        self.im = partie_imaginaire
    
    def __repr__(self):
        return f'Complex({self.re}, {self.im})'

    def conjugate(self):
        zconj = Complex(self.re, -self.im)
        return zconj

    def __str__(self):
        if self.im>0:
            return f'{self.re} + {self.im}i'
        elif z.im==0:
            return f'{self.re}'
        else:
            return f'{self.re} - {-self.im}i'
        
    def __add__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            return Complex(self.re + other, self.im)
        if isinstance(other, Complex):
            return Complex(self.re + other.re, self.im + other.im)
        return NotImplemented
        
    def __radd__(self, other):
        return self + other
    
    def affiche_miroir(self):
        print(self.__str__()[::-1])
        
    def __neg__(self):
        return Complex(-self.re, -self.im)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __getitem__(self, key):
        if key==0:
            return self.re
        if key==1:
            return self.im
        raise Exception()
            
class GPS:
    def __str__(coord):
        return f'{coord.lat}° {coord.long}°'


In [145]:
z = Complex(1, 3)
print(z.conjugate())
-z

1 - 3i


Complex(-1, -3)

In [148]:
z[1]

3

In [129]:
def tnirp(obj):
    obj.affiche_miroir()
    


In [130]:
tnirp(z)

i3 + 1


In [125]:
z1 = Complex(1, 3)
z2 = Complex(3.1, 6.3)

z1 + z2

Complex(4.1, 9.3)

In [131]:
def ajouter(obj1, obj2):
    resultat = obj1.__add__(obj2)
    if resultat==NotImplemented:
        resultat = obj2.__radd__(obj1)
    if resultat==NotImplemented:
        raise Exception(f'Je ne peux pas ajoute {obj1} à {obj2}')
    return resultat

add(3, z1)

Complex(4, 3)

In [ ]:
x = (1 + 3) * 6
prod(sum(1, 3), 6)

In [126]:
z1 + 3

Complex(4, 3)

In [127]:
3 + z1

Complex(4, 3)

In [122]:
print(z)

1 + 3i


In [96]:
def mon_print(obj):
    print(obj.__str__())
    
mon_print(z)

1 + 3i


In [97]:
print(z)

1 + 3i


In [91]:
z

Complex(1, 3)

In [83]:
s = "Bonjour\nHello"
s

'Bonjour\nHello'

In [84]:
print(s)

Bonjour
Hello


In [80]:
import numpy as np

a = np.random.rand(3)
print(a)
a

[0.40930193 0.14921632 0.22132264]


array([0.40930193, 0.14921632, 0.22132264])

In [82]:
np.array([0.40930193, 0.14921632, 0.22132264])

array([0.40930193, 0.14921632, 0.22132264])

### Dataclass

In [149]:
import dataclasses

@dataclasses.dataclass
class Book():
    titre : str
    auteur : str
    annee : int
        
book1 = Book("A very nice book", "F. Dupont", 2014)
book1

Book(titre='A very nice book', auteur='F. Dupont', annee=2014)

In [151]:
book1.titre

'A very nice book'

### Instrumentation

Exemple : oscilloscope (c.f. TP)

Inconvénients de la méthode fonctionnelle : 

* L'utilisateur doit importer beaucoup de fonction
* Les fonctions dépendent de l'instrument, il faut donc connaitre l'instrument pour savoir quelle fonction utiliser
* Il faut gérer les arguments des fonctions.

In [33]:
from fake_scpi import FakeSCPI
inst = FakeSCPI()

def set_frequency(inst, freq=1000):
    command = f'FREQ {freq:.3f}'
    inst.write(command)

def get_frequency(inst):
    command = f'FREQ?'
    return float(inst.query(command))

def get_model_identification(inst):
    res = inst.query("*IDN?")
    comp, model, serial, _ = res.split(',') 
    dic = {'compagnie':comp,
          'model':model,
          'serial':serial}
    return dic

set_frequency(inst, 100)
get_frequency(inst)
get_model_identification(inst)

{'compagnie': 'TEKTRONIX', 'model': 'DPO3014', 'serial': 'C012048'}

In [46]:
class Instrument():
    def __init__(self, conn):
        self._conn = conn
        
    def get_model_identification(self):
        res = self._conn.query("*IDN?")
        comp, model, serial, _ = res.split(',') 
        dic = {'compagnie':comp,
              'model':model,
              'serial':serial}
        return dic
    
    @property
    def idn(self):
        return self.get_model_identification()

class GBF(Instrument):
    frequency = property(lambda self:self.get_frequency(), lambda self, val:self.set_frequency(val))
    
    def get_frequency(self):
        raise Exception()
    
class TektronixGBF(GBF):
    def set_frequency(self, freq=1000):
        command = f'FREQ {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FREQ?'
        return float(self._conn.query(command))
    
        
class AgilentGBF(GBF):
    def set_frequency(self, freq=1000):
        command = f'FRE {freq:.3f}'
        self._conn.write(command)

    def get_frequency(self):
        command = f'FRE?'
        return float(self._conn.query(command))


    
gbf = TektronixGBF(FakeSCPI())

In [47]:
isinstance(gbf, GBF)

True

In [48]:
gbf.frequency = 10
gbf.frequency

10.0

## Nouveaux concepts



### Méthodes spéciales

- `__init__`
- `__repr__`, `__str__`

Lorsque c'est possible, `__repr__` doit représenté au mieux l'objet. Souvent il s'agit d'une chaine de caractère qui en étant évaluée renvoie un objet similaire. La méthode `__str__` renvoie `__repr__` par défaut. Sinon, elle doit être plus simple.

Opérateur unaire et binaire

- `__neg__`
- `__add__`, `__sub__`, `__mul__`, `__truediv__`, `__mod__`, `__pow__`
- `__radd__`, ...
- `__eq__` (==), `__ne__` (!=), `__lt__` (<), `__le__` (<=), `__gt__`, `__ge__`
- `__or__` (|), `__and__` (&), `__xor__`(^)
- `__matmul__` (@)


Emulateur de contenu

- a[key] => `a.__getitem__(key)`
- a[key] = val => `a.__setitem__(key, val)`
- del a[key] => `a.__delitem__(key)`
- len(a) => `a.__len__()`
- for elm in a => `for elm in a.__iter__()`

Emulateur de fonction 

- obj(a, b) => `obj.__call__(a, b)`

### Attributs et property

- Attributs de class vs attributs d'objets
- property

### Héritage

- isinstance permet de tester si un objet est une instance d'une classe. 

2

## Descripteur

Mieux comprendre les méthodes et les property...

<code>
    descr.__get__(self, obj, type=None) -> value
    descr.__set__(self, obj, value) -> None
    descr.__delete__(self, obj) -> None
</code>

Si un atribut de la classe défini la méthode `__get__` alors il devient un descripteur. Cette méthode est appelé à la place de l'attribut. 

In [111]:
#    def set_frequency(self, freq=1000):
#        command = f'FREQ {freq:.3f}'
#        self._conn.write(command)

#    def get_frequency(self):
#        command = f'FREQ?'
#        return float(self._conn.query(command))



class Descripteur:
    def __init__(self, param):
        self.param = param
    
    def __get__(self, obj, type=None):
        command = f'{self.param}?'
        return float(obj._conn.query(command))

    def __set__(self, obj, val):
        command = f'{self.param} {val:.3f}'
        obj._conn.write(command)

    
    
class Instrument():
    def __init__(self, conn):
        self._conn = conn
        
    frequency = Descripteur('FREQ')
    amplitude = Descripteur('AMP')

    
gbf = Instrument(FakeSCPI())
gbf.frequency = 12
print(gbf.frequency)

12.0


## Créer une class directement / metaclass

* On peut créer une classe avec la fonction type : `type(name, bases, namespace)`

* On peut remplacer cette fonction avec l'argument optionnel metaclass : 

<code>
    class MyClass(metaclass=type):
        pass
</code>

In [29]:
t = Test()

In [32]:
t.get_a()

10

In [31]:
t._a = 10